# FORCE ESTIMATION PREPROCESSING

## Specify Paths and FLAGS

In [ ]:
## TODO
# - unit convert to do but raw embedded into SI units (Si Machine) - DONE
# - jacobian preprocessing using interpolate jacobian (Si Machine) - DONE
# - split embeded data into force sensor ground truth (Si Machine ) - DONE
# - model prediction
# - procesing rosbag data through simons repo dvrk_force_estimation repo (Si Machine )
# - - rosbag reprocess (Si Machine)
# - - preprocess script(interpolations) (Si Machine)


# NOTES
#   make sure you put the json converted files in the jhu-davinci folders  

# file struture
# ---------------
# dataset_name 
# |__joints


## EMBEDDED PATHS
dataset_path = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/grip_fixture_low_force/" # SET
dataset_name = "grip_fixture_low_forcee"

fpga_v3_data_collection_path = "/home/ndrakes1/fpgav3-data-collection" # SET THIS ONCE
preproces_path = " "  # should be this directory SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_script_py = fpga_v3_data_collection_path + "/unit_convert/unit_convert.py"
si_unit_xml_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM3-698534.xml" # SET THIS ONCE 
si_unit_json_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-292409.xml.json"

zynq_data_csv = "/home/ndrakes1/data_collection_test/host/build/bin/capture_2-17-26/10khz/grip_fixture_low_force/grip_fixture_low_force.csv"

unit_convert_path = dataset_path + "/" + dataset_name + "_unitConvert.csv"



ORIGINAL_FREQ = 10000

INTERPOLATE = True


FILTER = True
FILTER_FREQ = 450
FILTER_VELOCITY = True
FILTER_POSITION = True

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 1000
DOWNSAMPLE_MOVING_AVERAGE = True

##################################################################################

# EMBEDDED

In [2]:
!. ~/catkin_ws/devel/setup.bash
from downsample import downsample_dataframe
import pandas as pd 
import os

## Unit Convert

In [3]:
!.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_csv}

Traceback (most recent call last):
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 115, in <module>
    main()
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 110, in main
    dataConverted = unitConvert(args.configuration, args.file_name)
  File "/home/ndrakes1/fpgav3-data-collection/unit_convert/unit_convert.py", line 15, in unitConvert
    robot_data = np.genfromtxt(fileName, delimiter=",", dtype=None, names=True,encoding='utf-8')
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/numpy/lib/npyio.py", line 2245, in genfromtxt
    values = split_line(line)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/numpy/lib/_iotools.py", line 226, in __call__
    return self._handyman(_decode_line(line, self.encoding))
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/numpy/lib/_iotools.py", li

## Extract Force Sensor Data

In [ ]:
!mkdir {dataset_path}/sensor
!.venv/bin/python3 sensor_processing.py {zynq_data_csv} {dataset_path}/sensor/sensor.csv
import filter

df = pd.read_csv( os.path.join(dataset_path,"sensor","sensor.csv") , header=None)

### FILTERING Force Sensor 

if (FILTER):

    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)
    df = filter.apply_filter_to_fs_df(df, fir_coeffs)

if (DOWNSAMPLE):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    df.to_csv(os.path.join(dataset_path,"sensor","sensor.csv"), index=False, header=False)



Saved SENSOR DATA CSV to /home/ndrakes1/force_estimation_data_captures/capture_2-5-26/grip_fixture_high_force//sensor/sensor.csv


AttributeError: type object 'filter' has no attribute 'design_fir_filter'

## Extract Joint Data

In [ ]:
!mkdir {dataset_path}joints
!.venv/bin/python3 preprocessing.py {unit_convert_path} {dataset_path}joints/interpolated_all_joints.csv

## Filter Joint Data

### Preprocessed Training File Paths

In [ ]:
import os
import pandas as pd

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")
df = pd.read_csv(data_file, header=None)


### Interpolate

In [ ]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):
    
    df = interpolate_dataframe_to_sample_rate(df, ORIGINAL_FREQ)


[0.00000000e+00 1.12260000e-04 1.84650000e-04 ... 1.21624644e+02
 1.21624644e+02 1.21624644e+02]


### FIR Filtering

In [ ]:
import importlib
import filter  # regular import first
importlib.reload(filter)


if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to data
    df = filter.apply_filter_to_torque_feedback_df(df, fir_coeffs, filter_velocity=FILTER_VELOCITY, filter_position=FILTER_POSITION)

### Downsample


In [ ]:


if (DOWNSAMPLE == True):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


### SAVE TO CSV

In [ ]:
df.to_csv(data_file, index=False, header=False)


## Extract Jacobian

In [ ]:
!mkdir {dataset_path}jacobian

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")

!.venv/bin/python3 interpolate_jacobian2.py {data_file} {dataset_path}jacobian/interpolated_all_jacobian.csv dvpsm.rob


FIRST JACOBIAN VALUE
[[ 1.38777878e-17  6.93889390e-18 -4.39038453e-03  0.00000000e+00
   1.75747326e-02 -1.32227720e-01]
 [-1.73472348e-18  3.46944695e-18  7.14793788e-01 -1.51788304e-18
   8.66046953e-02  1.27166121e-02]
 [ 0.00000000e+00  0.00000000e+00 -6.99321503e-01 -3.03576608e-18
   8.84104637e-02  1.38280974e-02]
 [ 2.77555756e-17 -9.99980294e-01  0.00000000e+00 -4.39038453e-03
  -9.90057470e-01 -1.39569383e-01]
 [-1.00000000e+00 -5.55111512e-17  0.00000000e+00  7.14793788e-01
   9.52158658e-02 -7.56588886e-01]
 [ 1.11022302e-16  6.27793940e-03  0.00000000e+00 -6.99321503e-01
   1.03538132e-01 -6.38822076e-01]]
Flattened Jacobians (column-major) written to /home/ndrakes1/force_estimation_data_captures/capture_1-28-26/contact_200Hz/grip_fixture_small_force/jacobian/interpolated_all_jacobian.csv


In [ ]:
# REMOVING JACOBIAN FILTERING -> MATHEMATICALLY INCORRECT


# df = pd.read_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), header=None)

# if DOWNSAMPLE == True:
#     df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
#     df.to_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), index=False, header=False)

# NOW DO INFERENCE ON THE JOINT DATA ON THE DVRK-12-MACHINE